# Description

- Initialize with returns, alpha, and spread
- Evaluate portfolios generated from the alpha

- TODO(gp): This should be called `Master_strategy_evaluator` like the class

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import logging

import core.config as cconfig
import core.dataflow_model.model_evaluator as modeval
import core.dataflow_model.utils as cdmu
import helpers.dbg as dbg
import helpers.printing as hprint

SyntaxError: EOL while scanning string literal (model_evaluator.py, line 138)

In [ ]:
dbg.init_logger(verbosity=logging.INFO)
# dbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", env.get_system_signature()[0])

hprint.config_notebook()

# Notebook config

In [1]:
# Read from env var.
eval_config = cconfig.Config.from_env_var("AM_CONFIG_CODE")

if config is None:
    experiment_dir = ""
    aws_profile = None
    selected_idxs = None

    eval_config = cconfig.get_config_from_nested_dict(
        {
            "load_experiment_kwargs": {
                "src_dir": experiment_dir,
                "file_name": "result_bundle.v2_0.pkl",
                "experiment_type": "ins_oos",
                "selected_idxs": selected_idxs,
                "aws_profile": aws_profile,
            },
            "strategy_evaluator_kwargs": {
                "returns_col": "mid_ret_0",
                "position_intent_col": "position_intent_1",
                "spread_col": "spread",
                "abort_on_error": True,
            },
            "bh_adj_threshold": 0.1,
            "resample_rule": "W",
        }
    )

print(str(eval_config))

NameError: name 'cconfig' is not defined

In [ ]:
# Build the ModelEvaluator from the eval config.
evaluator = modeval.StrategyEvaluator.from_eval_config(eval_config)

In [ ]:
# TODO(gp): Finish this.
keys = None
evaluator.compute_stats(keys)